In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from scipy.spatial.distance import cosine
from torch.utils.data import DataLoader
from pymystem3 import Mystem
import itertools
import pandas as pd
import numpy as np



/home/arizenelm/Cloud/Projects/word_distance/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model = SentenceTransformer('distiluse-base-multilingual-cased-v1')



/home/arizenelm/Cloud/Projects/word_distance/env/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:

df_match = pd.read_csv('data/match.csv', sep=';')
df_match.insert(len(df_match.columns), 'similarity', pd.NA)
df_match.insert(len(df_match.columns), 'similarity trained', pd.NA)

for i in range(len(df_match)) :
    embeddings = model.encode([df_match['nord'][i], df_match['rus'][i]])
    df_match.loc[i, 'similarity'] = 1 - cosine(embeddings[0], embeddings[1])


In [4]:
df_nordic = pd.read_csv('data/nordic.csv', sep=';')
df_russian = pd.read_csv('data/russian.csv', sep=';')


среднее для 100х100 -- 0.1227

In [5]:

match_nr = {df_match['nord'][i] : df_match['rus'][i] 
         for i in np.random.permutation(len(df_match))[:int(len(df_match) / 2)]}
match_rn = {df_match['rus'][i] : df_match['nord'][i] 
         for i in range(len(df_match))}

random_russian = np.random.permutation(df_russian['word'])

random_nordic = np.random.permutation(df_nordic['word'])

unmatch = []
for i, j in itertools.product(range(len(random_nordic)), range(len(random_russian))) :
    word = random_russian[j]
    if word not in match_rn or match_rn[word] != random_nordic[i] :
        unmatch.append((word, random_nordic[i]))
    if len(unmatch) == 100:
        break

print(len(unmatch))



100


In [6]:



train_examples = [InputExample(texts=[key, match_nr[key]], label=1) for key in match_nr]
train_examples = train_examples + [InputExample(texts=[x[0], x[1]], label=0) for x in unmatch]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
#train_loss = losses.CosineSimilarityLoss(model)
train_loss = losses.ContrastiveLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          epochs=2,
          warmup_steps=100,
          show_progress_bar=True,
          output_path='./models/model_trained')


Epoch:   0%|          | 0/2 [00:40<?, ?it/s]


KeyboardInterrupt: 